# Graphical Models
The central question that graphical models try to answer is how to more compactly write the joint distribution $P(X_1, X_2, ..., X_n)$ by accounting for the conditional independence relationships between variables. This is referred to as "factoring" the joint distribution. To emphasize the utility of doing so, consider that representing the full joint distribution (necessary when each variables always depends on all other variables) requires representing exponentially many states. In practice, variables may have some independence structure or relatively limited interaction with one another. Taking advantage of these conditions enables us to represent the joint distribution more or less equivalently with significantly fewer parameters. We will review a few common examples of how this is done, and then go over a problem involving a specific Bayesian Network. While there are many types of graphical models, we will only focus on directed graphical models whose structure is a directed acyclic graph.

### Naive Bayes
Suppose that we are considering how to classify vectors $X \in \{0, 1\}^d$ into binary labels $Y \in \{0, 1\}$. In general, we are trying to model the distribution $P(Y|X_1,X_2,...,X_d)$ which can be related to the joint distribution via $P(X_1,X_2,...,X_d,Y) = P(Y|X_1,X_2,...,X_d)P(X_1,X_2,...,X_d)$. However, Naive Bayes relies on a particular conditional independence assumption, namely that the features are conditionally independent given the label. This assumption allows us to write the conditional distribution of $X|Y$ as $P(X_1,X_2,...,X_d|Y) = \prod_{i=1}^d P(X_i|Y)$. The graphical model corresponding to Naive Bayes is as follows.

<img src="figures/naive_bayes.png">

While the Naive Bayes assumption is unlikely to exactly hold, it gives us a model that is tractable to represent (requiring only $2n + 1$ variables in the given binary case) and fairly robust in practice.

### Hidden Markov Models (HMMs)
Here, we'll discuss another type of graphical model commonly used for modeling sequences of data. Suppose instead that we have a sequence $X \in \{0, 1\}^n$ (unobserved "hidden" variables that we are trying to model) which produces an observable binary output $Y_i$ at each step $i$ conditioned on all previous information. We thus have a sequence of prediction problems based on the following joint distribution, which we will later show how to simplify.

$$P(X_1,X_2,...,X_n,Y_1,Y_2,...,Y_n) = \prod_{i=1}^n P(X_i, Y_i|X_{i-1}, Y_{i-1}, ...,X_1, Y_1)$$

As we can see, the states of this model depend on all past data, which becomes larger with every step. A common simplification (such as that adopted by n-gram models for natural language) is to reduce that dependence to a fixed window of size K. We will show how HMMs significantly reduce the problem space by only using a window of size 1 and adding a Markov assumption on the observed outputs.

First, we can apply the Markov assumption - that $X_i$ is dependent only on $X_{i=1}$ - to simplify the probability distribution of $P(X_i|X_{i-1}, Y_{i-1}...,X_1, Y_1) = P(X_i|X_{i-1})$. Next, we extend this assumption to $Y$ by making $Y_i$ conditionally independent of all previous information given $X_i$, and hence $P(Y_i|X_i,X_{i-1}, Y_{i-1}, ...,X_1, Y_1) = P(Y_i|X_i)$. The graphical model now takes on the following form.

<img src="figures/hmm.png">

The joint distribution thus simplifies to $P(X_1,X_2,...,X_n,Y_1,Y_2,...,Y_n) = \prod_{i=1}^n P(Y_i|X_i)P(X_i|X_{i-1})$. Inference can then be performed in linear time using a dynamic programming.

### Bayesian Networks and Conditional Independence Rules
Consider the some conditional independence tests involving the two following Bayes Nets.
<img src="figures/ci_example1.png">
#### Are A and B independent? (A $\perp$ B?) 
Yes.

#### Are A and B conditionally independent given C? (A $\perp$ B $|$ C?)
No. Because both A and B influence C, knowing B and C affects the distribution of A.

<img src="figures/ci_example2.png">

#### Are E and F independent? (E $\perp$ F?) 
No. Knowing F affects the possible values of D, which affects $P(E|D)$.

#### Are E and F conditionally independent given D? (E $\perp$ F $|$ D?)
Yes. The values of E and F only depend on $P(E|D)$ and $P(F|D)$.


There are two main rules that this example illustrates. The first involves the relationship between nodes and parents. The value of a node can be affected by conditioning on any descendent of its parents, because observing a parent's descendent can provide information about the parent. However, if the parent's value is already known, then the parent's non-shared descendents will have no effect on the given node. Finally, conditioning on all of a node's parents will make it conditionally independent of all of its non-descendents.  

The second rule involves the relationship between nodes and their children. If the only observed node is one of the node's descendents then it becomes dependent on all ancestors of the observed node. This is because all ancestors of a node affect the value of the observed node. Suppose we restrict the discussion to only the node's immediate children. Observing any of its children will make the node dependent on their children's other ancestors. However, the effect can be limited the node's children are also observed. 

Putting these rules together helps us derive the concept of a Markov Blanket. Given the values of a node's parents, the values of its children and its children's parents, the node will be conditionally independent of all other nodes in the graph, which also forms the basis of Gibbs Sampling.

<img src="figures/bayes_net.png">

#### What is the Markov Blanket of Alarm?
This includes all nodes except for Report, which is not needed because Alarm is conditionally independent of Report given Earthquake.

### Example: K-ZigZag Network

A $k$-$zigzag$ network has $k$ Boolean root variables and $k + 1$ Boolean leaf variables, where root $i$ is connected to leaves $i$ and $i + 1$. Here is an example for $k = 3$, where each $D_i$ represents a Boolean disease variable and each $S_j$ is a Boolean symptom variable:

<img src="figures/k_zigzag.png">

#### Does having symptom $S_4$ affect the probability of disease $D_1$? Why or why not?
No, because $S_4$ is not a descendant of $D_1$, and $D_1$ is independent of its nondescendants given its parents. 

#### Using only conditional probabilities from the model, express the probability of having symptom $S_1$ but not symptom $S_2$, given disease $D_1$.
$P(S_1, \neg S_2|D_1) = P(S_1|D_1)P(\neg S_2|D_1) = P(S_1|D_1)\sum_{d2} P(d_2)P(S_2| D_1, d_2)$

#### $True/False$: Exact inference in a k-zigzag net can be done in time O(k).
True, one can simply traverse the $2k - 1$ nodes one at a time and sum over the two possible states. 

#### Suppose the values of all the symptom variables have been observed, and you would like to do Gibbs sampling on the disease variables (i.e., sample each variable given its Markov blanket). What is the largest number of nonevidence variables that have to be considered when sampling any particular disease variable? Explain your answer

2 non-evidence variables - the Markov blanket consists of its children (2 evidence variables) and their other parents (2 non-evidence variables)

## Expectation Maximization (EM)
So far, we've mainly been concerned with the question of how to perform inference when the graph and the underlying probabilities are already known. We now turn our attention to the task of learning the probability tables for a known graph structure from data. In particular, we consider the case where the data that we're learning from contains missing entries. It is fairly straightforward to learn these tables when the data is not missing entries; one simply populates the tables with the empirical averages via the principle of maximum of likelihood. How do we properly compute these averages with missing data? At a high level, the solution adopted by EM is to assign preliminary (partial) values to the missing entries, evaluate the resulting expected likelihood, and then iteratively re-assign these partial values to optimize the likelihood. Let's define the problem formally.

The components of EM include a set of observed variables $X = \{X_1, \ldots, X_n\}$, a set of unobserved (latent) variables $Z = \{Z_1, \ldots, Z_n\}$ that we would like to infer, as well as a set of parameters $\Theta$ that we're trying to estimate. In the context of Bayes Nets, the $Z$ values are the partial assignments of the unlabeled data, and $\Theta$ the parameters of the probability tables that we would like to learn. Consider also a clustering example (which we will go over in this recitation) where $Z$ are the assignments of the data to clusters and the $\Theta$ parameters are the centroids of each cluster. The EM algorithm iterates on the following two steps.
   1. First, during the expectation phase, we calculate the partial assignments $Z$ based on the known parameters and use them to compute the expected log-likehood, $\mathbb{E}_{P(Z|X, \Theta)}[\log P(X, Z | \Theta)]$.
   2. Then, during the maximization phase, we compute new values of the parameters $\Theta$ that maximize the inner expected log-likelihood term, setting the parameters to $\Theta = argmax_{\Theta'} \, \mathbb{E}_{P(Z|X, \Theta)}[\log P(X, Z | \Theta')]$.
Let's explore these steps in greater detail with a clustering example.

Consider the K-Means problem, which is the problem of assigning each data point to one of $K$ different clusters such that the overall squared distance between data points and their assigned cluster mean is minimized.
$$ \min_{Z, \Theta} \sum_{i=1}^n \sum_{j=1}^K \mathbf{1}[z_i == K] \|x_i - \theta_i\|^2_2$$
There is a simple greedy algorithm for K-Means (known as Lloyd's algorithm) which assigns each point to its closest cluster, recomputes the cluster centroids, and then repeats these steps until convergence. We will show that by modeling clusters as Gaussians of unit covariance, we can use the framework of EM to derive a "soft" version of the classic K-means algorithm that uses partial assignments instead of hard assignments.

Suppose that we have observed data points $X \in \mathbb{R}^{n \times d}$ that have an associated set of "latent" assignment variables $Z \in \{1, \ldots, K\}^n$ and cluster centroid parameters $\Theta \in \mathbb{R}^{k \times d}$. 
Assume that the likelihood of $x_i$ belong to a given cluster $k$ is a multivariate Gaussian with mean $\theta_k$ and unit covariance.
$$P(x_i, z_i = k | \Theta) = \frac{1}{(2\pi)^\frac{d}{2}\|I\|^2_2} \exp\left(-\frac{1}{2}(x_i - \theta_i)^T I^{-1} (x_i - \theta_i)\right) = \frac{1}{(2\pi)^\frac{d}{2}} \exp\left(-\frac{1}{2}\|x_i - \theta_i\|^2_2\right)$$
We will now walk through the two EM steps in detail. For the expectation step, note that because the $x_i$'s and $z_i$'s corresponding to data point $i$ are independent of those of the other data points, we can rewrite $P(Z_1,\ldots,Z_n|X_1,\ldots,X_n, \Theta) = \prod_{i=1}^n P(Z_i|X_i, \Theta)$ and $\log(P(X_1, Z_1,\ldots,X_n,Z_n | \Theta)) = \sum_{i=1}^n \log(P(X_i, Z_i | \Theta))$. First we will compute the partial assignment of each $Z_i$ using the definition of conditional probability.

$$\begin{align} P(z_i = k|x_i, \Theta) &= \frac{P(x_i, z_i = k | \Theta)}{\sum_{j=1}^K P(x_i, z_i = j | \Theta)} \\ &= \frac{\frac{1}{(2\pi)^\frac{d}{2}} \exp\left(-\frac{1}{2}\|x_i - \theta_k\|^2_2\right)}{\sum_{j=1}^K \frac{1}{(2\pi)^\frac{d}{2}} \exp\left(-\frac{1}{2}\|x_i - \theta_j\|^2_2\right)}  \\ &= \frac{\exp\left(-\frac{1}{2}\|x_i - \theta_k\|^2_2\right)}{\sum_{j=1}^K \exp\left(-\frac{1}{2}\|x_i - \theta_j\|^2_2\right)} \\ &= \delta(z_i = k, x_i, \Theta) \end{align}$$

Note the resemblance to the softmax function. Because these terms remain fixed during the maximization step, we will denote them as $\delta(z_i = k, x_i, \Theta)$. We then use the partial assignments of $z_i$ to compute the expected log-likelihood.

$$\begin{align} \mathbb{E}_{P(Z|X, \Theta)}[\log P(X, Z | \Theta)] &= \sum_{i=1}^n \sum_{k=1}^K \delta(z_i = k, x_i, \Theta) \log P(x_i, z_i = k | \Theta) \\ &= \sum_{i=1}^n \sum_{k=1}^K \delta(z_i = k, x_i, \Theta) \left(-\frac{d}{2} \log(2\pi) + \frac{1}{2}\|x_i - \theta_k\|^2_2 \right) \end{align}$$

Once we have this expression for the expected log-likelihood, we then optimize each of the cluster centroid by taking a mean over the points "partially assigned" to each cluster; essentially, this is a weighted average where the weights for each point $x_i$ are $\delta(z_i = k, x_i, \Theta)$.

$$\frac{\partial \mathbb{E}_{P(Z|X, \Theta)}[\log P(X, Z | \Theta)]}{\partial \theta_k} = \sum_{i=1}^n \delta(z_i = k, x_i, \Theta) (x_i - \theta_k) = 0$$
$$\theta_k = \frac{\sum_{i=1}^n \delta(z_i = k, x_i, \Theta) x_i}{\sum_{i=1}^n \delta(z_i = k, x_i, \Theta)} $$

We will wrap up this discussion with a toy EM clustering problem. Suppose that have 4 1-D data points that we are trying to cluster into two clusters. $X = \{0, 1, 4, 5\}$. We initialize $\Theta = \{2, 3\}$.  Show the results of performing one round of expectation maximization, including the partial assignments of $Z$, the expression for the log-likelihood , and the recomputed parameters.

## Reinforcement Learning
The high-level goal of reinforcement learning is to learn a policy $\Pi$ such that an agent taking actions based on this policy optimizes a reward function in a given environment. Here, we will review some of the basics of reinforcement learning, such as the definition of a Markov Decision Process, illustrate this with some examples, and show how this formulation naturally leads to the concepts of policy and value iteration.

### Markov Decision Processes